In [ ]:
# default_exp troubleshooting

In [ ]:
#hide
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#hide
!pip install nbdev

In [ ]:
#hide
%cd /content/drive/My\ Drive/nbd_colab

/content/drive/My Drive/nbd_colab


In [ ]:
#export
from nbdev import *

# Troubleshooting

> There are a number of quircks and gotchas to be aware of when working with nbdev from within Colaboratory. When you encounter a problem check the following. Many of them may seem obvious but can still trip you up and waste time - they have me!



### 1. Make sure you are in the project directory.

* Library and module imports as well as nbdev commands only work from within the project directory. Make sure you %cd 
into the project directory before you start working in a notebook. Place a `%cd path_to_project_directory` command in a early cell (after installs but before any imports) so as not to forget! 

* System errors can reset notebooks and return the current working directory to home (i.e. '/content'). Thus, following a system error, particularly in the project management notebook if you use one, always check that you are still in the project directory before continuing to work (`!pwd`).

* Oh, and make sure to use `%cd` not `!cd` from within notebooks to change directory!


### 2. Local imports not working.

* Managing local imports between Colab and Google Drive can be a little frustrating but a systematic approach saves the 
day. 

* For a new import, or a change to the code of an existing import, to become available in the importing notebook the following are required: 

 * first, the project must be rebuilt with `nbdev_build_lib`. This creates the a new or updated .py file containing the actual code to be imported.

 * second, Colab may not realise an updated import has changed and so doesn't import the the new .py file when the !import statement is rerun. If this occurs, try restarting the runtime ('restart runtime...' in the notebook's runtime menu) and/or reloading the page. If this doesn't work - and more often than not it doesn't - do a 'factory reset runtime'. This resets all variables and forces all imports to be reloaded from source. Factory resets are a pain (you have to re-mount Google Drive and reinstall any external libraries) but I've found it's possible to plan project development so as to minimise changes to imported code and reduce time wasted in this way.
 
* If the import is still not working, check the source notebook's corresponding .py file. Is the new code there? Did you `#export` it from the source notebook?. If so, repeat the build then factory reset the destination notebook BEFORE trying to import it again. 

### 3. `Show_doc` doesn't give the expected notebook cell output.
* The nbdev `show_doc(class.method)` used to automatically generate documentation for class methods doesn't give the expected text output in Colab notebook cells, instead just a method definition. However, it works as expected when the documentation is actually built with `!nbdev_build_docs` so continue to use it as per the nbdev docs/tutorial (https://nbdev.fast.ai/).

### 4. `nbdev_build_docs` fails with an error message.

* Most often, it is a failed `!import` in one of the notebooks. Make sure the import is working in the notebook itself (see above if not) and if so rebuild the project. If the error persists and it is not obvious why, reset the runtime in the project management notebook (a factory reset may be required).

### 4. `nbdev_diff_nbs` generates an output.

* Code diffs between the notebooks and build files can cause problems with github integration. If nbdev_diff_nbs gives an output, therefore, you'll need to either run `notebook2script()` or `script2notebook()` from a notebook cell to fix the difference (see the nbdev docs at https://nbdev.fast.ai/). 

### 5. `nbdev_test_nbs` fails.

* Testing is great and `nbdev_test_nbs` is an incredibly easy way of implementing them in your notebooks and across a whole project. I add tests to just about anything in my notebooks that can be tested and this saves a lot of time and frustration, especially as projects grow in size. Tests can be an excellent way of showing users how to use your code and can be included in the docs. Those you don't want exposed to the user `#hide`. 

* As well as simply reading the error message and debugging your code, it is worth considering the folowing when `nbdev_test_nbs` fails:

* For `!nbdev_test_nbs` to give a clean bill of health, each notebook in the project (including index.ipynb) must run independently from top to bottom without stalling or raising an exception. It is worth doing this for all your notebooks before building and testing.

* If you can't find the cause of an exception from the error message, identify the failing notebook, do a factory reset then rerun it top to bottom including remounting Google Drive and installing libraries. This will usually identify the cause of the error and allow debugging. Once the culprit notebook is running top to bottom error free - rebuild the project library and docs and rerun `!nbdev_test_nbs`. 

* Note that `drive.mount()` in an open unmounted notebook will cause the notebook to stall awaiting user imput and therefore tests to fail. Either save and close the notebook session or run the cell and mount Drive so that the code runs through without hitch next time. 

* `!nbdev_test_nbs` reruns the nbs on a colab CPU instance. Thus all device code (i.e. running Learner.fit() in fastai) will cause a `cuda runtime error (100)`. I havn't found a way round this yet so while ignoring it is fine, and doesn't intefere with git integration or pypi make release, it may mask errors further down in the notebook. So make sure you can run the notebook top to bottom on a CUDA enabled Colab instance when this error arises.  

### 6. GitHub integration issues.

* GitHUB integration works almost exactly the same as when used from a local machine command line. Just mmake sure you have activated !`nbdev_git_hooks` to strip notebooks of unwanted metadata and that the local repo is configured with your GitHub username and email:  
```
! git config user.email "email"
! git config user.name "username"
```

* Not infrequently, however, allthough all the nbdev tests passed, git push fails with a little red cross. You can click on the red cross for more details. Common causes are: 

 * an unresolved diff. Go back and run `!nbdev git hooks` in the project directory and try again.

 * the code to mount Google Drive, `from google.colab import Drive` will raise a 'ModuleNotFoundError: No module named 'google' exception when nbdev_test_nbs is run during github tests. I assume this is because git tests are run on a server without google.colab installed. No way round this for Colab users - comment out the offending code if you really want the green tick next to your repo on GithHub but otherwise, it doesn't make a difference to anything. 

* Investigate and debug other causes of failing tests during a GitHub push, but there shouldn't really be any if `!nbdev_test_nbs` was successfully run prior to pushing.

### 7. Documentation on GitHub Pages. 

* Github pages only shows the index.ipynb page without any other notebook documentation. Make sure you have checked the 'master branch/docs folder' option of 'GitHub Pages' (in the repo settings). The 'docs' folder is ignored if the 'master branch' only option is selected 

* A notebook doesn't appear in the docs sidebar menu. The menu is built from the first `#` markdown cell of each notebook, where the `#` must be followed by a unique notebook name e.g `# Core`. Include this before any other markdown cells and the notebook will appear in the menu under this name. 

* Otherwise, if the docs don't look as expected on GitHub Pages, revise the nbdev syntax for auto-generating documentaion from notebook cells and review your code. 